**Aim**: Build interactive plots to visulize gene-level phenotypes.

In [1]:
!pip uninstall -y screenpro2
!pip install git+https://github.com/arcinstitute/screenpro2.git@dev

Found existing installation: ScreenPro2 0.4.17
Uninstalling ScreenPro2-0.4.17:
  Successfully uninstalled ScreenPro2-0.4.17
  Cloning https://github.com/arcinstitute/screenpro2.git (to revision dev) to /private/var/folders/2l/9947v345357ch7g4kkxgcbjr0000gp/T/pip-req-build-9s84f23o
  Running command git clone --filter=blob:none --quiet https://github.com/arcinstitute/screenpro2.git /private/var/folders/2l/9947v345357ch7g4kkxgcbjr0000gp/T/pip-req-build-9s84f23o
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  branch 'dev' set up to track 'origin/dev'.
  Resolved https://github.com/arcinstitute/screenpro2.git to commit 3a54987a815dbe73b0fb5c565842953b10e38202
  Preparing metadata (setup.py) ... done
  Created wheel for ScreenPro2: filename=ScreenPro2-0.4.18-py3-none-any.whl size=43533 sha256=edd37ff200332415ecb93a8ecdb5fa97119ed19dafce485b5b743992fcc74883
  Stored in directory: /private/var/folders/2l/9947v345357ch7g4kkxgcbjr0000gp/T/pip-ephem-wh

In [11]:
!cd ../

In [12]:
pwd

'/Users/abearab/Projects/FunGI-cap/CRISPRScreenDB/DDRi'

In [13]:
import screenpro as scp
import pandas as pd
import numpy as np
import pygwalker as pyg
import bokeh

In [3]:
drug_names = {
    'DMSO': 'DMSO', # Vehicle
    'Pi': 'PARPi',
    'Ri': 'ATRi',
    'Wi': 'WEE1i',
    'Mi': 'ATMi',
    'Ki': 'DNAPKi',
    'PiRi': 'PARPi+ATRi',
    'PiWi': 'PARPi+WEE1i',
    'PiMi': 'PARPi+ATMi',
    'PiKi': 'PARPi+DNAPKi'
}

### A549_CRISPRi_v2_screens

In [14]:
A549_CRISPRi_v2_screens = scp.load._read_screen_pkl('screens/A549_CRISPRi_v2_screens')

In [15]:
A549_CRISPRi_v2_results = A549_CRISPRi_v2_screens.phenotypes['compare_guides']['results']

___

In [16]:
A549_CRISPRi_v2_screens.phenotypes['compare_guides']['results'].keys()

dict_keys(['gamma:DMSO_vs_T0', 'tau:Pi_vs_T0', 'rho:Pi_vs_DMSO', 'tau:Ri_vs_T0', 'rho:Ri_vs_DMSO', 'tau:PiRi_vs_T0', 'rho:PiRi_vs_DMSO', 'tau:Mi_vs_T0', 'rho:Mi_vs_DMSO', 'tau:PiMi_vs_T0', 'rho:PiMi_vs_DMSO', 'tau:Wi_vs_T0', 'rho:Wi_vs_DMSO', 'tau:PiWi_vs_T0', 'rho:PiWi_vs_DMSO', 'tau:Ki_vs_T0', 'rho:Ki_vs_DMSO', 'tau:PiKi_vs_T0', 'rho:PiKi_vs_DMSO'])

In [18]:
!mkdir -p app/_html/A549_CRISPRi_v2_screens

for drug_key in list(drug_names.keys())[1:]:

    drug = drug_names[drug_key]
    print(drug)

    plotter = scp.dashboard.DrugScreenDashboard(
        A549_CRISPRi_v2_screens,
        untreated='DMSO',
        treated=drug_key,
        run_name='compare_guides',
        threshold=2,
        ctrl_label='negative_control',
        pvalue_col='ttest pvalue',
    )

    rho_min = plotter.df['rho_score'].min() + 0.01
    rho_max = plotter.df['rho_score'].max() - 0.01
    gamma_min = plotter.df['gamma_score'].min() + 0.01
    gamma_max = plotter.df['gamma_score'].max() - 0.01

    p_vol = plotter.RhoVolcanoPlot(
        x_source='rho_score', 
        y_source='-log10(rho_pvalue)', 
        xaxis_label='CRISPRi drug phenotype (ρ)',
        yaxis_label='-log10(p-value)',
        up_hit='resistance_hit', down_hit='sensitivity_hit', 
        hit_label_col='rho_label', 
        x_min=rho_min, x_max=rho_max,
        y_min=0.1, y_max='auto',
        dot_size=5,
        return_html=True
    )

    p_sct = plotter.RhoGammaScatter(
        x_source='rho_score', 
        y_source='gamma_score', 
        xaxis_label='CRISPRi drug phenotype (ρ)',
        yaxis_label='CRISPRi control phenotype (γ)', 
        up_hit='resistance_hit', down_hit='sensitivity_hit', 
        hit_label_col='rho_label', 
        x_min=rho_min, x_max=rho_max,
        y_min=gamma_min, y_max=gamma_max,
        legend_loc=None,
        dot_size=5,
        return_html=True
    )

    # write plots to html files
    with open(f'app/data/A549_CRISPRi_v2_screens/{drug}_volcano.html', 'w') as f:
        f.write(p_vol)

    with open(f'app/data/A549_CRISPRi_v2_screens/{drug}_scatter.html', 'w') as f:
        f.write(p_sct)

    plotter.df.query(
        'rho_label in ["resistance_hit", "sensitivity_hit"]'
    ).to_csv(f'app/data/A549_CRISPRi_v2_screens/{drug}_volcano.csv', index=False)

PARPi


/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p1048', ...) are being repeated
  p = bokeh.plotting.figure(
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p1176', ...) are being repeated
  p = bokeh.plotting.figure(


ATRi


/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p1304', ...) are being repeated
  p = bokeh.plotting.figure(
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p1432', ...) are being repeated
  p = bokeh.plotting.figure(


WEE1i


...

/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p2584', ...) are being repeated
  p = bokeh.plotting.figure(
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p2712', ...) are being repeated
  p = bokeh.plotting.figure(


PARPi+ATMi


/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p2840', ...) are being repeated
  p = bokeh.plotting.figure(
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p2968', ...) are being repeated
  p = bokeh.plotting.figure(


PARPi+DNAPKi


/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p3096', ...) are being repeated
  p = bokeh.plotting.figure(
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p3224', ...) are being repeated
  p = bokeh.plotting.figure(


### A549_CRISPRa_v2_screens

In [13]:
# A549_CRISPRa_v2_screens = scp.load._read_screen_pkl('screens/A549_CRISPRa_v2_screens')

In [14]:
# A549_CRISPRa_v2_results = A549_CRISPRa_v2_screens.phenotypes['compare_guides']['results']

___

In [15]:
# A549_CRISPRa_v2_screens.phenotypes['compare_guides']['results'].keys()

dict_keys(['gamma:DMSO_vs_T0', 'tau:Pi_vs_T0', 'rho:Pi_vs_DMSO', 'tau:Ri_vs_T0', 'rho:Ri_vs_DMSO', 'tau:PiRi_vs_T0', 'rho:PiRi_vs_DMSO', 'tau:Wi_vs_T0', 'rho:Wi_vs_DMSO', 'tau:PiWi_vs_T0', 'rho:PiWi_vs_DMSO'])

In [16]:
# A549_CRISPRa_v2_keys = [
#     k.split(':')[1].split('_')[0] 
#     for k in A549_CRISPRa_v2_screens.phenotypes['compare_guides']['results'].keys()
#     if 'rho' in k
# ]

In [18]:
# plots = {}

# !mkdir -p app/_html/A549_CRISPRa_v2_screens/

# for drug_key in list(drug_names.keys())[1:]:
#     if drug_key not in A549_CRISPRa_v2_keys: break

#     print(drug_key)

#     plotter = scp.dashboard.DrugScreenDashboard(
#         A549_CRISPRa_v2_screens,
#         untreated='DMSO',
#         treated=drug_key,
#         run_name='compare_guides',
#         threshold=2,
#         ctrl_label='negative_control',
#         pvalue_col='ttest pvalue',
#     )

#     p = plotter.RhoVolcanoPlot(
#         x_source='rho_score', 
#         y_source='-log10(rho_pvalue)', 
#         xaxis_label='phenotype score', 
#         yaxis_label='-log10(p-value)', 
#         up_hit='resistance_hit', down_hit='sensitivity_hit', 
#         hit_label_col='rho_label', 
#         x_min=-2.5, x_max=2.5, 
#         y_min=0, y_max='auto', 
#         return_html=True
#     )
   
#     plotter.df.query(
#         'rho_label in ["resistance_hit", "sensitivity_hit"]'
#     ).to_csv(f'app/data/A549_CRISPRa_v2_screens/{drug_names[drug_key]}_volcano.csv')

#     plots[drug_names[drug_key]] = p


# # write plots to html files
# for drug, html in plots.items():
#     with open(f'app/data/A549_CRISPRa_v2_screens/{drug}_volcano.html', 'w') as f:
#         f.write(html)

Pi


/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p3736', ...) are being repeated
  p = bokeh.plotting.figure(
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p3864', ...) are being repeated
  p = bokeh.plotting.figure(


Ri
Wi


/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/abearab/miniconda3/envs/screenpro2/lib/python3.11/site-packages/screenpro/dashboard/__init__.py:23: UserWarning: ResetTool(id='p3992', ...) are being repeated
  p = bokeh.plotting.figure(


### A549_PRDX1_CRISPRi_v3_screens

In [94]:
# A549_PRDX1_CRISPRi_v3_screens.phenotypes

(1) compare_reps_parent

In [95]:
# A549_PRDX1_CRISPRi_v3_parent_results = A549_PRDX1_CRISPRi_v3_screens.phenotypes['compare_reps_parent']['results']

___

In [96]:
# A549_PRDX1_CRISPRi_v3_screens.phenotypes['compare_reps_vehicle']['results'].keys()

In [97]:
# A549_PRDX1_CRISPRi_v3_screens.phenotypes['compare_reps_parent']['results'].keys()

In [98]:
# plots = {}

# plotter = scp.dashboard.DrugScreenDashboard(
#     A549_CRISPRa_v2_screens,
#     untreated='DMSO',
#     treated=drug_key,
#     run_name='compare_guides',
#     threshold=2,
#     ctrl_label='negative_control',
#     pvalue_col='ttest pvalue',
# )

# p = plotter.RhoVolcanoPlot(
#     x_source='rho_score', 
#     y_source='-log10(rho_pvalue)', 
#     xaxis_label='phenotype score', 
#     yaxis_label='-log10(p-value)', 
#     up_hit='resistance_hit', down_hit='sensitivity_hit', 
#     hit_label_col='rho_label', 
#     x_min=-2.5, x_max=2.5, 
#     y_min=0, y_max='auto', 
#     return_html=True
# )

# plots[f'DNAPKi_{subscreen}']


# !mkdir app/_html/A549_PRDX1_CRISPRi_v3_screens/

# # write plots to html files
# for drug, html in plots.items():
#     with open(f'app/_html/A549_PRDX1_CRISPRi_v3_screens/{drug}_{}_volcano.html', 'w') as f:
#         f.write(html)

In [76]:
# from IPython.display import IFrame

In [75]:
# IFrame(src='app/_html/A549_CRISPRi_v2_screens/PARPi_volcano.html', width=800, height=600)

#

In [21]:
from watermark import watermark
print(
    watermark()
)

Last updated: 2025-01-04T18:25:06.016227-08:00

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 8.27.0

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [22]:
print(
    watermark(iversions=True, globals_=globals())
)

bokeh    : 3.6.0
numpy    : 1.26.4
screenpro: 0.4.18
pygwalker: 0.4.9.13
pandas   : 2.2.3
watermark: 2.5.0



___